In [206]:
%matplotlib inline
import matplotlib.pyplot as plt
import sympy as sp 
import numpy as np
import pylab
sp.init_printing()

In [207]:
l1, l2 = sp.symbols('l1 l2', real=True)
q1, q2 = sp.physics.vector.dynamicsymbols('q1 q2')
m1, m2 = sp.symbols('m1 m2', real=True)
I1_11, I1_12, I1_21, I1_22 = sp.symbols('I1_11 I1_12 I1_21 I1_22', real=True)
I2_11, I2_12, I2_21, I2_22 = sp.symbols('I2_11 I2_12 I2_21 I2_22', real=True)
I1 = sp.Matrix([[I1_11, I1_12], [I1_21, I1_22]])
I2 = sp.Matrix([[I2_11, I2_12], [I2_21, I2_22]])

In [208]:
x1 = l1 * sp.cos(q1)
y1 = l1 * sp.sin(q1)
X1 = sp.Matrix([x1, y1])
X1

⎡l₁⋅cos(q₁(t))⎤
⎢             ⎥
⎣l₁⋅sin(q₁(t))⎦

In [209]:
x = x1 + l2 * sp.cos(q1 + q2)
y = y1 + l2 * sp.sin(q1 + q2)
X = sp.Matrix([x, y])
X

⎡l₁⋅cos(q₁(t)) + l₂⋅cos(q₁(t) + q₂(t))⎤
⎢                                     ⎥
⎣l₁⋅sin(q₁(t)) + l₂⋅sin(q₁(t) + q₂(t))⎦

In [210]:
J1 = X1.jacobian([q1, q2])
J1

⎡-l₁⋅sin(q₁(t))  0⎤
⎢                 ⎥
⎣l₁⋅cos(q₁(t))   0⎦

In [211]:
J = X.jacobian([q1, q2])
J

⎡-l₁⋅sin(q₁(t)) - l₂⋅sin(q₁(t) + q₂(t))  -l₂⋅sin(q₁(t) + q₂(t))⎤
⎢                                                              ⎥
⎣l₁⋅cos(q₁(t)) + l₂⋅cos(q₁(t) + q₂(t))   l₂⋅cos(q₁(t) + q₂(t)) ⎦

In [212]:
q = sp.Matrix([q1, q2])
q

⎡q₁(t)⎤
⎢     ⎥
⎣q₂(t)⎦

In [213]:
q_dot = sp.diff(q)
q_dot

⎡d        ⎤
⎢──(q₁(t))⎥
⎢dt       ⎥
⎢         ⎥
⎢d        ⎥
⎢──(q₂(t))⎥
⎣dt       ⎦

In [214]:
I = I1 @ sp.Matrix([[1, 0], [0, 0]]) + I2
I

⎡I₁ ₁₁ + I₂ ₁₁  I₂ ₁₂⎤
⎢                    ⎥
⎣I₁ ₂₁ + I₂ ₂₁  I₂ ₂₂⎦

# Inertia matrix

In [215]:
M = m1 @ J1.T @ J1 + m2 @ J.T @ J + I
M = sp.simplify(M)
M

⎡                  2        2                                2                
⎢I₁ ₁₁ + I₂ ₁₁ + l₁ ⋅m₁ + l₁ ⋅m₂ + 2⋅l₁⋅l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂  I₂ ₁₂ + l₁⋅
⎢                                                                             
⎢                                                  2                          
⎣          I₁ ₂₁ + I₂ ₂₁ + l₁⋅l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂                       

                     2   ⎤
l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂⎥
                         ⎥
          2              ⎥
I₂ ₂₂ + l₂ ⋅m₂           ⎦

# Kinetic energy

In [216]:
K = 0.5 @ q_dot.T @ M @ q_dot
K

⎡⎛    ⎛          2   ⎞ d               ⎛                                2   ⎞ 
⎢⎜0.5⋅⎝I₂ ₂₂ + l₂ ⋅m₂⎠⋅──(q₂(t)) + 0.5⋅⎝I₂ ₁₂ + l₁⋅l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂⎠⋅
⎣⎝                     dt                                                     

d        ⎞ d           ⎛    ⎛                                        2   ⎞ d  
──(q₁(t))⎟⋅──(q₂(t)) + ⎜0.5⋅⎝I₁ ₂₁ + I₂ ₂₁ + l₁⋅l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂⎠⋅──(
dt       ⎠ dt          ⎝                                                   dt 

             ⎛                  2        2                                2   
q₂(t)) + 0.5⋅⎝I₁ ₁₁ + I₂ ₁₁ + l₁ ⋅m₁ + l₁ ⋅m₂ + 2⋅l₁⋅l₂⋅m₂⋅cos(q₂(t)) + l₂ ⋅m₂
                                                                              

⎞ d        ⎞ d        ⎤
⎠⋅──(q₁(t))⎟⋅──(q₁(t))⎥
  dt       ⎠ dt       ⎦

# Potential energy

In [217]:
U = sp.Matrix([0])
U

[0]

# Coriolis matrix

In [218]:
def christoffel_symbols(M, i, j, k):
    return 0.5 * (sp.diff(M[j,k], q[i]) + sp.diff(M[k,i], q[j]) - sp.diff(M[i,j], q[k]))

c111 = christoffel_symbols(M, 0, 0, 0)
c112 = christoffel_symbols(M, 0, 0, 1)
c121 = christoffel_symbols(M, 0, 1, 0)
c122 = christoffel_symbols(M, 0, 1, 1)
c211 = christoffel_symbols(M, 1, 0, 0)
c212 = christoffel_symbols(M, 1, 0, 1)
c221 = christoffel_symbols(M, 1, 1, 0)
c222 = christoffel_symbols(M, 1, 1, 1)

C = sp.Matrix([[c111 + c211, c112 + c212], [c121 + c221, c122 + c222]])
C

⎡-1.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))  1.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))⎤
⎢                                                 ⎥
⎣-2.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))             0           ⎦

# Lagrange equations

In [219]:
#tau1, tau2 = sp.symbols('tau_1 tau_2', real=True)
#tau = sp.Matrix([tau1, tau2])

In [220]:
q_ddot = sp.diff(q_dot)
q_ddot

⎡  2       ⎤
⎢ d        ⎥
⎢───(q₁(t))⎥
⎢  2       ⎥
⎢dt        ⎥
⎢          ⎥
⎢  2       ⎥
⎢ d        ⎥
⎢───(q₂(t))⎥
⎢  2       ⎥
⎣dt        ⎦

In [221]:
tau = M @ q_ddot + C @ q_dot
tau

⎡                                                                             
⎢                          d                                   d           ⎛  
⎢- 1.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))⋅──(q₁(t)) + 1.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))⋅──(q₂(t)) + ⎝I₂
⎢                          dt                                  dt             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                 d           
⎢                                       - 2.0⋅l₁⋅l₂⋅m₂⋅sin(q₂(t))⋅──(q₁(t)) + 
⎢                                                                 dt          
⎣                                                                             

                                      2                                       
                              2   ⎞  d           ⎛ 